In [3]:
def simple_coroutine(): #
    print('-> coroutine started')
    x = yield#
    print('-> coroutine received:', x)

In [4]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x10ca7b480>

In [5]:
next(my_coro)

-> coroutine started


In [6]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [9]:

def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)


In [10]:
my_coro2 = simple_coro2(14)

In [11]:
from inspect import getgeneratorstate

In [12]:
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [13]:
next(my_coro2)

-> Started: a = 14


14

In [14]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [15]:
my_coro2.send(28)

-> Received: b = 28


42

In [16]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [17]:
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count

In [18]:
coro_avg = averager()

In [19]:
next(coro_avg)

In [20]:
coro_avg.send(10)

10.0

In [21]:
coro_avg.send(30)

20.0

In [22]:
coro_avg.send(5)

15.0

In [2]:
from functools import wraps

def coroutine(func):
    """Decorator: primes `func` by advancing to first `yield`""" 
    @wraps(func)
    def primer(*args,**kwargs):
        gen = func(*args,**kwargs) 
        next(gen)
        return gen
    return primer

In [3]:
@coroutine
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count

In [26]:
coro_avg = averager()

In [27]:
from inspect import getgeneratorstate

In [28]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [29]:
coro_avg.send(10)

10.0

In [30]:
coro_avg.send(30)

20.0

In [31]:
coro_avg.send(5)

15.0

In [4]:
# How an unhandled exception kills a coroutine.
coro_avg = averager()

In [5]:
coro_avg.send(40)

40.0

In [6]:
coro_avg.send(50)

45.0

In [7]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [8]:
coro_avg.send(60)

StopIteration: 

In [9]:
#  code for an averager coroutine that returns a re‐ sult.
from collections import namedtuple
Result = namedtuple('Result', 'count average')
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [11]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [7]:
# using yield from to drive averager and report statistic.
from collections import namedtuple

Result = namedtuple('Result', 'count average')
# the subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# the delegating generator
def grouper(results, key): 
    while True:
        results[key] = yield from averager()

# the client code, a.k.a. the caller
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key) 
        next(group)
        for value in values:
            group.send(value) 
        group.send(None) # important!
    print(results)  # uncomment to debug
    report(results) 

# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))
        
data = { 'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [8]:
import collections
import queue
import random

Event = collections.namedtuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):
    """Yield to simulator issuing event at each state change""" 
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger') 
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')

def compute_duration(action):
    return random.randint(1, 10)

class Simulator:
    def __init__(self, procs_map): 
        self.events = queue.PriorityQueue() 
        self.procs = dict(procs_map)
        
    def run(self, end_time):
        """Schedule and display events until time is up""" # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
        # main loop of the simulation
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty(): 
                print('*** end of events ***')
                break
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event 
            print('taxi:',proc_id,proc_id*' ',current_event) 
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action) 
            try:
                next_event = active_proc.send(next_time) 
            except StopIteration:
                del self.procs[proc_id] 
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***' 
            print(msg.format(self.events.qsize()))
DEPARTURE_INTERVAL = 5
num_taxis = 3
taxis = {i: taxi_process(i, (i + 1) * 2, i * DEPARTURE_INTERVAL) for i in range(num_taxis)}
sim = Simulator(taxis)
sim.run(30)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 1   Event(time=5, proc=1, action='leave garage')
taxi: 1   Event(time=6, proc=1, action='pick up passenger')
taxi: 0  Event(time=10, proc=0, action='pick up passenger')
taxi: 2    Event(time=10, proc=2, action='leave garage')
taxi: 0  Event(time=12, proc=0, action='drop off passenger')
taxi: 0  Event(time=13, proc=0, action='pick up passenger')
taxi: 0  Event(time=14, proc=0, action='drop off passenger')
taxi: 1   Event(time=14, proc=1, action='drop off passenger')
taxi: 2    Event(time=15, proc=2, action='pick up passenger')
taxi: 0  Event(time=16, proc=0, action='going home')
taxi: 1   Event(time=16, proc=1, action='pick up passenger')
taxi: 1   Event(time=20, proc=1, action='drop off passenger')
taxi: 2    Event(time=21, proc=2, action='drop off passenger')
taxi: 1   Event(time=25, proc=1, action='pick up passenger')
taxi: 2    Event(time=25, proc=2, action='pick up passenger')
taxi: 2    Event(time=30, proc=2, action='drop